In [1]:
import csv
import json
import math as m
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import datetime as dt
import yfinance as yf
import seaborn as sb
from datetime import datetime, timedelta
from functools import reduce
import yahoo_fin.stock_info as si
from yahoo_fin import *
import jinja2
import tradingview_ta
from tradingview_ta import TA_Handler, Interval, Exchange
from yahoofinancials import YahooFinancials
from stocksymbol import StockSymbol
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier
from pypfopt import plotting
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import EfficientCVaR
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
plt.style.use('bmh')

In [2]:
# created a csv file of ticker symbols for an excel database
nasdaq = pd.DataFrame(si.tickers_nasdaq())
sym = set( symbol for symbol in nasdaq[0].values.tolist() )

# deleting the inactive tickers by suffix
my_list = ['W', 'R', 'P', 'Q']

# variable for tickers to be saved and disgarded
del_set = set()
sav_set = set()

# loop to id the suffix and disgard
for symbol in sym:
    if len( symbol ) > 4 and symbol[-1] in my_list:
        del_set.add( symbol )
    else:
        sav_set.add( symbol )

In [5]:
yf.pdr_override()

# retrieving the ticker list from S&P 500 w/o typing the ticker symbols
assets = si.tickers_nasdaq()
financial = YahooFinancials(sav_set)

# creating date variables going back one month to today's date
begin = (dt.datetime.today() - dt.timedelta(30)).strftime('%Y-%m-%d')
stop = dt.datetime.today().strftime('%Y-%m-%d')

# retrieving historical data through Yahoo Financials
data = financial.get_historical_price_data(begin, stop, 'daily')

# creating a database for Adjusted Close on every ticker symbol
prices = pd.DataFrame({
    a: {x['formatted_date']: x['adjclose'] for x in data[a]['prices']} for a in assets
})

# naming index column
prices.index.name = 'Date'
prices

TypeError: string indices must be integers

In [4]:
prices = pd.read_csv('prices.csv',index_col=0,parse_dates=True, encoding='utf-8')
prices

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 30071: invalid start byte